In [ ]:
!pip install fake-useragent

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 5.7 MB/s eta 0:00:00


In [ ]:
import os
import requests
import re
from bs4 import BeautifulSoup
from pprint import pprint
from fake_useragent import UserAgent
from tqdm.auto import tqdm

In [ ]:
# проверка на адекватность
session = requests.session()
ua = UserAgent()
headers = {'User-Agent': ua.random}
response = session.get('http://www.dlib.si/results/?query=%27rel%3dknjige%40OR%40rel%3dvisoko%C5%A1olska+dela%27&sortDir=ASC&sort=date&flanguage=slv&ftype=knjige&faccess=public&pageSize=100&page=1&fUDC=Knji%C5%BEevnost', headers=headers)
print(response.status_code)
response = session.get('https://www.dlib.si/results/?query=%27rel%3dknjige%40OR%40rel%3dvisoko%c5%a1olska+dela%27&flanguage=slv&ftype=knjige&sortDir=ASC&sort=date&frights=InC&pageSize=100&page=1')
print(response.status_code)

200
200


In [ ]:
if not os.path.exists('texts'):
    os.makedirs('texts')

In [ ]:
# Base URL pattern
base_url = "https://www.dlib.si/results/?query=%27rel%3dknjige%40OR%40rel%3dvisoko%c5%a1olska+dela%27&ftype=knjige&flanguage=slv&frights=PDM&pageSize=100&page={}"


# Process all 64 pages
for page in tqdm(range(1, 64), desc="Processing pages"):
    url = base_url.format(page)
    response = session.get(url, headers=headers)

    if response.status_code != 200:
        print(f"Failed to fetch page {page}")
        continue

    soup = BeautifulSoup(response.content, 'html.parser')
    records = soup.find_all('div', class_='record')

    for record in tqdm(records, desc=f"Page {page} records", leave=False):
        try:
            # Extract title from Naslov class
            title_element = record.find('div', class_='Naslov').find('b')
            title = title_element.text.strip() if title_element else "Unknown Title"

            # Clean title from invalid characters
            title = re.sub(r'[\\/*?:"<>|]', '', title)
            if len(title) > 100:  # If too long
                title = title[:100] + "..."

            # Extract author from Avtorji class
            avtorji_element = record.find('div', class_='Avtorji')
            if avtorji_element:
                author_links = avtorji_element.find_all('a')
                if author_links:
                    first_author = author_links[0].text.strip()
                    author = re.sub(r'\s*\(.*\)', '', first_author)
                else:
                    author = "Unknown Author"
            else:
                author = "Unknown Author"

            # Clean author from invalid characters
            author = re.sub(r'[\\/*?:"<>|]', '', author)
            if len(author) > 50:  # # If too long
                author = author[:50] + "..."

            # Find all links to texts
            txt_links = record.find_all('a', href=lambda href: href and 'TEXT' in href)

            for i, link in enumerate(txt_links, start=1):
                full_url = 'http://www.dlib.si' + link['href']

                file_response = session.get(full_url, headers=headers)

                if file_response.status_code == 200:
                    # Filename in "title (author).txt" format
                    if len(txt_links) > 1:
                        filename = f"{title} ({author}) - {i}.txt"
                    else:
                        filename = f"{title} ({author}).txt"

                    save_path = os.path.join('texts', filename)

                    # Handle duplicates
                    if os.path.exists(save_path):
                        # Add suffix if exists
                        base, ext = os.path.splitext(save_path)
                        counter = 1
                        while os.path.exists(f"{base}_{counter}{ext}"):
                            counter += 1
                        save_path = f"{base}_{counter}{ext}"

                    # Save the file
                    with open(save_path, 'wb') as f:
                        f.write(file_response.content)

                    print(f"Downloaded: {save_path}")
                else:
                    print(f"Failed to download from {full_url}")

        except Exception as e:
            print(f"Error processing record on page {page}: {e}")
            continue

print("All pages processed successfully!")

Processing pages:   0%|          | 0/63 [00:00<?, ?it/s]

Page 1 records:   0%|          | 0/100 [00:00<?, ?it/s]

Downloaded: texts/Kronika Pevskega zbora Glasbene matice v Ljubljani 1891-1941 (Mahkota, Karel).txt
Downloaded: texts/Katóljşhki nauk od sakramentov svéte pokóre in presvétiga réshnjiga telésa (Švab, Gašper).txt
Downloaded: texts/Gluh mora biti (Moinaux, Jules).txt
Downloaded: texts/Kratki navuki, regelze, inu molitve sa sakrament té pokore. (Debevec, Janez).txt
Downloaded: texts/Poročilo o kmetijski enketi dne 17. in 18. aprila 1884 v Ljubljani (Vošnjak, Josip).txt
Downloaded: texts/Národno blágo vogrskij Sloväncof (Kühar, Števan).txt
Downloaded: texts/Kozarec vode (Scribe, Eugene).txt
Downloaded: texts/Koroške bukvice (Haderlap, Filip) - 1.txt
Downloaded: texts/Koroške bukvice (Haderlap, Filip) - 2.txt
Downloaded: texts/Nemško-slovenska pravna terminologija (Babnik, Janko).txt
Downloaded: texts/Ena lepa lubesniva inu brania vredna historia od te po nadoushnu ven isgnane svete grafnie Genofe... (Unknown Author).txt
Downloaded: texts/Katalog okrajne učiteljske knjižnice v Črnomlju 1911

Page 2 records:   0%|          | 0/100 [00:00<?, ?it/s]

Downloaded: texts/Luzh, inu senza vofra s. mashe, ali Molitve per s. mashi, po teh imenitnejsheh skryvnostah terple... (Pohlin, Marko).txt
Downloaded: texts/Zbrani spisi Pavline Pajkove (Pajk, Pavlina) - 1.txt
Downloaded: texts/Zbrani spisi Pavline Pajkove (Pajk, Pavlina) - 2.txt
Downloaded: texts/Križem sveta (Slemenik, Franc Valentin).txt
Downloaded: texts/Domači prepír (Zabukovec, Jakob).txt
Downloaded: texts/Znanstvena terminologija s posebnim ozirom na srednja učilišča (Cigale, Matej).txt
Downloaded: texts/Cerkvica sv. Mohora na Tolminskih Pečinah (Kaffol, Jakob Filip).txt
Downloaded: texts/Varh (Blaznik, Jožef).txt
Downloaded: texts/Kaj naj danes kuham (Unknown Author).txt
Downloaded: texts/Zbrani spisi. Izdal Anton Klodič-Sabladoski. (Pagliaruzzi, Josip).txt
Downloaded: texts/Rudninoslovje ali mineralogija za niže gimnazije in realke (Erjavec, Fran).txt
Downloaded: texts/Dva gospoda pa jeden sluga (Goldoni, Carlo).txt
Downloaded: texts/Kratko premišljęvánje Kristusovęga tęrpljen

...

Downloaded: texts/Zapisnik hiš in hišnih posestnikov deželnega stolnega mesta ljubljanskega (Robida, Ivan).txt
Downloaded: texts/Lehrplan für siebenclassige Volksschulen, an denen die zweite Landessprache als obligater Lehrgeg... (Unknown Author).txt
Downloaded: texts/Pesnik Aškerc v borbi za herojstvo (Tominšek, Josip).txt
Downloaded: texts/Vrla gospodinja (Rihar, Luitgarda).txt
Downloaded: texts/Slava Mariji (Unknown Author).txt
Downloaded: texts/Lehrplan für sechsclassige Volksschulen, an denen die zweite Landessprache als obligater Lehrgege... (Unknown Author).txt
Downloaded: texts/Jurija Dalmatina Predgovor k bibliji iz l. 1584 (Dalmatin, Jurij).txt
Downloaded: texts/Lehrplan für siebenklassige Volksschulen, an denen die zweite Landessprache als obligater Lehrgeg... (Unknown Author).txt
Downloaded: texts/Prečastitemu gospodu Josipu Domenisu v spomin njegove prve sv. maše slovesno darovane v Sovodnji ... (Trinko, Ivan).txt
Downloaded: texts/Začetnica za slovenske ljudske šole (Kopr

Page 62 records:   0%|          | 0/100 [00:00<?, ?it/s]

Downloaded: texts/Župnija in božja pot Device Marije v Puščavi (Pajek, Jožef).txt
Downloaded: texts/Maksimiljan I., cesar mehikanski (Nedeljko, Fran).txt
Downloaded: texts/Allgemeine Bergpolizei-Verordnung für den Bezirk des k. k. Revierbergamtes Cilli (Unknown Author).txt
Downloaded: texts/Slava Prešernu! (Gangl, Engelbert)_1.txt
Downloaded: texts/Božja pot Matere Božje na Blejskem jezeru (Čarman, Franciscus)_4.txt
Downloaded: texts/Humanizem gimnazij v stari in novi luči (Tominšek, Josip).txt
Downloaded: texts/Konsumna društva, njihov začetek in konec (Unknown Author).txt
Downloaded: texts/Strta mladost (Pavlič, Rado).txt
Downloaded: texts/Molítvi, na sztári szlovenszki jezik obrnyene, ino na haszek szlovenzkoga národa vö dáne po Szyár... (Szyarto, Stevan)_1.txt
Downloaded: texts/Robinzon starši (Defoe, Daniel).txt
Downloaded: texts/Zgodbe Sv. pisma za nižje razrede ljudskih šol (Knecht, Friedrich Justus)_1.txt
Downloaded: texts/Črtice (Baloh, Ivan).txt
Downloaded: texts/Zemljepis za

Page 63 records:   0%|          | 0/100 [00:00<?, ?it/s]

Downloaded: texts/Ljudska kuharica (Račič, Krista).txt
Downloaded: texts/Kuharica v kmečki, delavski in preprosti meščanski hiši (Remec, Marija).txt
Downloaded: texts/Dijaški koledar za navadno leto 1893 (Unknown Author).txt
Downloaded: texts/Krfska deklaracija in demokracija (Kristan, Etbin).txt
Downloaded: texts/Računica za obče ljudske šole (Črnivec, Anton)_7.txt
Downloaded: texts/Dijaški koledar za navadno leto 1894 (Unknown Author).txt
Downloaded: texts/Naukapolne pripovéşti sa şlovénşko mladost (Zabukovšek, Jožef).txt
Downloaded: texts/Tilho in drugi (Kleinmayr, Ferdo).txt
Downloaded: texts/Semena padajo (Žgur, Fran).txt
Downloaded: texts/Svetopisemska knjiga Job in psalem ! 118 (Sedej, Frančišek Borgia).txt
Downloaded: texts/Slovenska slovstvena čitanka za sedmi in osmi razred srednjih šol (Sket, Jakob)_1.txt
Downloaded: texts/Pri severnih Slovanih (Lavtižar, Josip).txt
Downloaded: texts/Vojska na Daljnem Vzhodu (Lampe, Evgen).txt
Downloaded: texts/Spisi Andrejčkovega Jožeta (Po

In [ ]:
# Save results
!zip -r ./texts_new.zip ./texts/

Выходные данные были обрезаны до нескольких последних строк (5000).
  adding: texts/Zakon o kontroliranju čistine izdelkov iz zlata, srebra in platine (Unknown Author).txt (deflated 64%)
  adding: texts/Malo berilo za slovenske ljudske šole (Unknown Author).txt (deflated 57%)
  adding: texts/Četrti zbornik poezij (Aškerc, Anton).txt (deflated 56%)
  adding: texts/Malý slovinsko-český slovník (Očenášek, Jindřich).txt (deflated 59%)
  adding: texts/Aritmetika za učiteljišča (Močnik, Franc).txt (deflated 61%)
  adding: texts/Sto cirkvenih ino drygih pobožnih pesmi med katolškimi kristjani slovenskega naroda na Štajarskem (Dajnko, Peter).txt (deflated 61%)
  adding: texts/Elektrika, nje proizvajanje in uporaba (Šubic, Ivan).txt (deflated 63%)
  adding: texts/Zlati orehi slovenskej mladini v spomin (Tomšič, Ivan).txt (deflated 58%)
  adding: texts/Sveto leto 1901 (Zamejic, Andrej).txt (deflated 61%)
  adding: texts/V proslavo petdesetletnega vladanja Nj. Veličanstva Franca Jožefa I. pririre